In [1]:
from itertools import repeat
from joblib import Parallel, delayed
import numpy as np
import os
from ava.preprocessing.preprocess import process_sylls, \
	tune_syll_preprocessing_params
from ava.preprocessing.utils import get_spec
from ava.segmenting.segment import tune_segmenting_params
from ava.segmenting.segment import segment
from ava.segmenting.amplitude_segmentation import get_onsets_offsets
from ava.data.data_container import DataContainer
from ava.models.vae_dataset import get_syllable_partition
from ava.segmenting.refine_segments import refine_segments_pre_vae
import matplotlib.pyplot as plt
import shutil

In [2]:
X_SHAPE = (128,128)
"""Processed spectrogram shape: ``[freq_bins, time_bins]``"""
X_DIM = np.prod(X_SHAPE)

In [3]:
#########################################
# 0) Define directories and parameters. #
#########################################
params = {
	'segment': {
		'min_freq': 30e3, # minimum frequency
		'max_freq': 110e3, # maximum frequency         # originally 110e3
		'nperseg': 1024, # FFT
		'noverlap': 512, # FFT
		'spec_min_val': 2.0, # minimum log-spectrogram value
		'spec_max_val': 6.0, # maximum log-spectrogram value
		'fs': 303030, # audio samplerate               #originally 250000
		'th_1':0.33, # segmenting threshold 1           #originally 0.1
		'th_2':0.37,# segmenting threshold 2           #originally 0.2
		'th_3':0.374, # segmenting threshold 2           #originally 0.3
		'max_dur': 0.2, # maximum syllable duration   #originally 0.2
		'min_dur':0.01, # minimum syllable duration    #originally 0.03
		'smoothing_timescale': 0.007, # timescale for smoothing amplitude trace
		'softmax': True, # puts amplitude values in [0,1]
		'temperature': 0.5, # temperature parameter for softmax
		'algorithm': get_onsets_offsets, # segmentation algorithm
	},
	'preprocess': {
		'get_spec': get_spec,
		'max_dur': 0.2, # maximum syllable duration    #originally 0.2
		'min_freq': 30e3, # minimum frequency
		'max_freq': 110e3, # maximum frequency         # originally 110e3
		'num_freq_bins': X_SHAPE[0], # hard-coded
		'num_time_bins': X_SHAPE[1], # hard-coded
		'nperseg': 1024, # FFT
		'noverlap': 512, # FFT
		'spec_min_val': 2.0, # minimum log-spectrogram value
		'spec_max_val': 6.0, # maximum log-spectrogram value
		'fs': 303030, # audio samplerate             #originally 250000 So we are samping every 3x10^-6 seconds
		'mel': False, # frequency spacing, mel or linear
		'time_stretch': True, # stretch short syllables?
		'within_syll_normalize': False, # normalize spectrogram values on a
										# spectrogram-by-spectrogram basis
		'max_num_syllables': None, # maximum number of syllables per directory
		'sylls_per_file': 20, # syllable per file
		'real_preprocess_params': ('min_freq', 'max_freq', 'spec_min_val', \
				'spec_max_val', 'max_dur'), # tunable parameters
		'int_preprocess_params': ('nperseg','noverlap'), # tunable parameters
		'binary_preprocess_params': ('time_stretch', 'mel', \
				'within_syll_normalize'), # tunable parameters
	},
}

#root = '/Users/achintkumar/autoencoded-vocal-analysis/My_codes/Sample_C57_USV_Files'
#root = '/hdd/achint_files/Sample_C57_USV_Files'

#audio_dirs = [os.path.join(root,'audio')] 
root = '/hdd/achint_files/achint_mousedata/Tom Data'
dataset_both      = ['Group1','Group2']
audio_dirs_group1 = [os.path.join(root,'Group1')]
seg_dirs_group1   = [os.path.join(root,'Group1','segs')]
proj_dirs_group1  = [os.path.join(root,'Group1','projections')]
spec_dirs_group1  = [os.path.join(root,'Group1','specs')]

audio_dirs_group2 = [os.path.join(root,'Group2')]
seg_dirs_group2   = [os.path.join(root,'Group2','segs')]
proj_dirs_group2  = [os.path.join(root,'Group2','projections')]
spec_dirs_group2  = [os.path.join(root, 'Group2','specs')]

audio_dirs_both   = [os.path.join(root,group) for group in dataset_both ]
seg_dirs_both     = [os.path.join(root,group,'segs') for group in dataset_both]
proj_dirs_both    = [os.path.join(root,group,'projections') for group in dataset_both]
spec_dirs_both    = [os.path.join(root,group,'specs') for group in dataset_both]

model_filename = os.path.join(root,'joint_template', 'checkpoint_150.tar')  # Originally xxx=150
plots_dir = os.path.join(root,'plots')
dc_1    = DataContainer(projection_dirs=proj_dirs_group1, spec_dirs=spec_dirs_group1, \
		plots_dir=plots_dir, model_filename=model_filename)
dc_2    = DataContainer(projection_dirs=proj_dirs_group2, spec_dirs=spec_dirs_group2, \
		plots_dir=plots_dir, model_filename=model_filename)
dc_both = DataContainer(projection_dirs=proj_dirs_both, spec_dirs=spec_dirs_both, \
		plots_dir=plots_dir, model_filename=model_filename)
##################################
# 1) Tune segmenting parameters. #
##################################
params['segment'] = tune_segmenting_params(audio_dirs_both, params['segment'])

# Here fs = 303030
# spec_min_val = 2
# spec_max_val = 7
# max_freq = fs/2=151515

Tune segmenting parameters
---------------------------


Set value for min_freq: [30000.0]  
Set value for max_freq: [110000.0]  
Set value for nperseg: [1024]  
Set value for noverlap: [512]  
Set value for spec_min_val: [2.0]  
Set value for spec_max_val: [6.0]  
Set value for fs: [303030]  
Set value for th_1: [0.33]  
Set value for th_2: [0.37]  
Set value for th_3: [0.374]  
Set value for max_dur: [0.2]  
Set value for min_dur: [0.01]  
Set value for smoothing_timescale: [0.007]  
Set value for temperature: [0.5]  
Continue? [y] or [s]top tuning or [r]etune params:  s


In [4]:
###############
# 2) Segment. #
###############
n_jobs = min(len(audio_dirs_both), os.cpu_count()-1)
gen = zip(audio_dirs_both, seg_dirs_both, repeat(params['segment']))
for data in gen:
    print("Reaching here")
    print(data)
    segment(*data)
# Parallel(n_jobs=n_jobs)(delayed(segment)(*args) for args in gen)

Reaching here
('/hdd/achint_files/achint_mousedata/Tom Data/Group1', '/hdd/achint_files/achint_mousedata/Tom Data/Group1/segs', {'min_freq': 30000.0, 'max_freq': 110000.0, 'nperseg': 1024, 'noverlap': 512, 'spec_min_val': 2.0, 'spec_max_val': 6.0, 'fs': 303030, 'th_1': 0.33, 'th_2': 0.37, 'th_3': 0.374, 'max_dur': 0.2, 'min_dur': 0.01, 'smoothing_timescale': 0.007, 'softmax': True, 'temperature': 0.5, 'algorithm': <function get_onsets_offsets at 0x7f16be6cc310>})
Segmenting audio in /hdd/achint_files/achint_mousedata/Tom Data/Group1
	Found 6112 segments in /hdd/achint_files/achint_mousedata/Tom Data/Group1
Reaching here
('/hdd/achint_files/achint_mousedata/Tom Data/Group2', '/hdd/achint_files/achint_mousedata/Tom Data/Group2/segs', {'min_freq': 30000.0, 'max_freq': 110000.0, 'nperseg': 1024, 'noverlap': 512, 'spec_min_val': 2.0, 'spec_max_val': 6.0, 'fs': 303030, 'th_1': 0.33, 'th_2': 0.37, 'th_3': 0.374, 'max_dur': 0.2, 'min_dur': 0.01, 'smoothing_timescale': 0.007, 'softmax': True, '

In [ ]:
#####################################
# 3) Tune preprocessing parameters. #
#####################################
preprocess_params = tune_syll_preprocessing_params(audio_dirs_both, seg_dirs_both, \
		params['preprocess'])
params['preprocess'] = preprocess_params

Tune preprocessing parameters:


In [ ]:
##################
# 4) Preprocess. #
##################
n_jobs = os.cpu_count()-1
gen = zip(audio_dirs_both, seg_dirs_both, spec_dirs_both, repeat(params['preprocess']))
Parallel(n_jobs=n_jobs)(delayed(process_sylls)(*args) for args in gen)